# Getting the embeddings

This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

> It uses sliding window view instead of sliding window for splitting the dataset.

## Set-up
Initial notebook setup and specific debugging and pre-configured cases selection.
### VsCode update patch
Initial notebook setup when using VSCode.

In [ ]:
# This is only needed if the notebook is run in VSCode
import sys
if '--vscode' in sys.argv:
    print("Executing inside vscode")
    import nbs_pipeline.utils.vscode  as vs
    vs.DisplayHandle.update = vs.update_patch

### Debugging variables

- `print_flag`. If `True` it adds debbuging messages in those functions that allows so (eg. `get_enc_embeddings`)
- `reset_kernel`. If `True` it resets the kernel by the end of the execution. Use only in case that memory management is needed.
- `check_memory_usage`. If `True`, it adds some lines for checking the GPU memmory ussage along the execution.
- `time_flag`. If `True` it get the execution time along the notebook as well as inside those functions that allows so (eg. `get_enc_embeddings`)

In [ ]:
print_flag = False
reset_kernel = True
check_memory_usage = True
time_flag = True

## Main code
### Import libraries

In [ ]:
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *
import wandb
from yaml import load, FullLoader
if check_memory_usage:
    import nbs_pipeline.utils.memory as mem
    import torch 
import utils.config as cfg

### Initialize and Configurate Artifact

In [ ]:
wandb_api = wandb.Api()

In [ ]:
if check_memory_usage:
    gpu_device = torch.cuda.current_device()
    mem.gpu_memory_status(gpu_device)

#### Get configuration parameters from yml
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

In [ ]:
config, job_type = cfg.get_artifact_config_embeddings(print_flag = print_flag)
if print_flag: cfg.show_attrdict(config)

### Setup W&B artifact

In [ ]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03b_embeddings-sliding_window_view"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

In [ ]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

### Get trained model artifact

##### Build artifact selector
Botch to use artifacts offline

In [ ]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

##### Get the model from W&B
Restore the encoder model and its associated configuration

In [ ]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [ ]:
if print_flag: enc_artifact.metadata

In [ ]:
print("enc_artifact: "+enc_artifact.name)

In [ ]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [ ]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [ ]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

In [ ]:
df = input_ar.to_df()

if print_flag: 
    display(df.head())
    print("df ~ ", df.shape)

### Split data with Sliding Window

In [ ]:
import time

In [ ]:
w = enc_run.config['w']
if print_flag: print(w)

In [ ]:
if check_memory_usage: mem.gpu_memory_status(gpu_device)

In [ ]:
type(df)

In [ ]:
if time_flag: t_start = time.time()

enc_input, _ = prepare_forecasting_data(df, fcst_history = w)

if time_flag: 
    t_end = time.time()
    t = t_end - t_start
    print("SW start | " , t_start, " | end ", t_end, "total (secs): ", t)

if print_flag: print(enc_input.shape)

In [ ]:
if check_memory_usage: mem.gpu_memory_status(gpu_device)

### Get embedings

In [ ]:
stride = enc_run.config['stride']
batch_size = enc_run.config['batch_size']
#enc_learner.dls.bs = enc_run.config['batch_size']

In [ ]:
if print_flag:
    print(stride)
    print(batch_size)
    print(enc_input.shape)
    print(enc_artifact.name)

In [ ]:
print(enc_input.shape)

In [ ]:
if check_memory_usage: mem.gpu_memory_status(gpu_device)

In [ ]:
if time_flag: t_start = time.time()

embs = get_enc_embs_set_stride_set_batch_size(
    X = enc_input, 
    enc_learn = enc_learner, 
    stride = stride, 
    batch_size = batch_size, 
    cpu=config.cpu, 
    to_numpy=True, 
    print_flag = print_flag,
    time_flag = time_flag,
    chunk_size = 1000,
    check_memory_usage = check_memory_usage
)
if time_flag:
    t_end = time.time()
    t = t_end - t_start
    print("GE start | " , t_start, " | end ", t_end, "total (secs): ", t)

In [ ]:
%debug

In [ ]:
if check_memory_usage: mem.gpu_memory_status(gpu_device)

In [ ]:
if config.use_wandb: 
    run.log_artifact(
        ReferenceArtifact(
            embs, 
            'embeddings-mvp-SWV', 
            metadata=dict(run.config), 
            type='embeddings'
        ),
        aliases=f'run-{run.project}-{run.id}'
    )

In [ ]:
run.finish()

## Final checks

### Check num_inputs = embs
num_inputs = len(enc_input[::stride]) = enc_input.shape[0]/stride + 1 if extra block = ceil(enc_input.shape[0]/stride

num_embs = embs.shape[0]

In [ ]:
#Dimensions check
num_inputs = np.ceil(enc_input.shape[0]/stride)
num_embs = embs.shape[0]
test_eq(num_inputs, num_embs )
print(num_inputs, num_embs)

# Free GPU

In [ ]:
if check_memory_usage: mem.gpu_memory_status(gpu_device)

In [ ]:
print("Execution ended")
if reset_kernel:
    import os
    os._exit(00)